In [1]:
import pandas as pd

# 파일 경로 설정
metal_binding_sites_path = "/home/qkrgangeun/LigMet/code/text/biolip/metal_binding_sites3_grouped.csv"
train_pdbs_path = "/home/qkrgangeun/LigMet/code/text/biolip/train_pdbs.txt"
output_path = "/home/qkrgangeun/LigMet/code/text/biolip/train_metal_binding_sites3.csv"

# metal_binding_sites3_grouped.csv 파일 읽기
df = pd.read_csv(metal_binding_sites_path)

# train_pdbs.txt 파일에서 pdb id 읽기
# 파일 내 각 줄에 pdb id가 하나씩 있다고 가정합니다.
with open(train_pdbs_path, "r") as f:
    train_pdb_list = [line.strip() for line in f if line.strip()]

# df에서 "PDB ID" 열의 값이 train_pdb_list에 포함된 행만 필터링
filtered_df = df[df["PDB ID"].isin(train_pdb_list)]

# 필터링한 결과를 train_metal_binding_sites.csv로 저장
filtered_df.to_csv(output_path, index=False)

print(f"총 {len(filtered_df)}개의 행이 {output_path}로 저장되었습니다.")


총 91765개의 행이 /home/qkrgangeun/LigMet/code/text/biolip/train_metal_binding_sites3.csv로 저장되었습니다.


In [2]:
import pandas as pd

# CSV 파일 경로 지정
csv_file = "/home/qkrgangeun/LigMet/code/text/biolip/train_metal_binding_sites3.csv"

# CSV 파일 읽어오기
df = pd.read_csv(csv_file)

# group_id 열의 고유(unique) 값 개수를 계산
unique_group_count = df["group_id"].nunique()

print("Unique group_id 개수:", unique_group_count)


Unique group_id 개수: 25273


In [3]:
import pandas as pd

# CSV 파일 경로 지정
csv_file = "/home/qkrgangeun/LigMet/code/text/biolip/train_metal_binding_sites3.csv"

# CSV 파일을 DataFrame으로 불러오기
df = pd.read_csv(csv_file)

# 각 group_id 별로 첫 번째로 나온 PDB ID와 pdb id 개수를 구하기
# 여기서는 "PDB ID" 열을 기준으로 그룹화한다고 가정합니다.
result = df.groupby("group_id").agg(
    representative=("PDB ID", "first"),
    pdb_count=("PDB ID", "count")
).reset_index()

# 결과 출력
print(result)


       group_id representative  pdb_count
0             0           2fv9          1
1             1           3fii          1
2             2           5jr6          1
3             3           1n51          1
4             4           5x49          2
...         ...            ...        ...
25268     28521           4r9x          1
25269     28522           4kjk          1
25270     28523           4rjl          2
25271     28524           3m6z          1
25272     28525           5a94         16

[25273 rows x 3 columns]


In [5]:
import pandas as pd

# CSV 파일 경로 지정
csv_file = "/home/qkrgangeun/LigMet/code/text/biolip/train_metal_binding_sites3.csv"
result_file = "/home/qkrgangeun/LigMet/code/text/biolip/rf_train_pdbs_group.txt"

# CSV 파일을 DataFrame으로 불러오기
df = pd.read_csv(csv_file)

# 각 group_id별로 첫 번째로 나온 PDB ID와 pdb id 개수를 구하기 (여기서는 "PDB ID" 열을 기준으로 그룹화)
result = df.groupby("group_id").agg(
    representative=("PDB ID", "first"),
    pdb_count=("PDB ID", "count")
).reset_index()

# 대표 pdb id 리스트 추출 ("representative" 컬럼)
pdb_ids = result["representative"].tolist()

# 대표 pdb id를 rf_train_pdbs_group.txt에 저장 (한 줄에 하나씩 기록)
with open(result_file, "w") as f:
    for pdb_id in pdb_ids:
        f.write(f"{pdb_id}\n")

print(f"대표 PDB ID가 {result_file}에 저장되었습니다.")


대표 PDB ID가 /home/qkrgangeun/LigMet/code/text/biolip/rf_train_pdbs_group.txt에 저장되었습니다.


In [4]:
import pandas as pd
import ast

# CSV 파일 경로 지정
csv_file = "/home/qkrgangeun/LigMet/code/text/biolip/train_metal_binding_sites3.csv"

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# 각 행의 "Binding Residues" 열에서 결합 잔기 개수를 계산하는 함수 정의
def count_binding_residues(x):
    """
    x: 문자열 형태의 binding residues 정보 (예: "('002', 'HIS', 'HIS', 'HIS')")
    반환: 결합 잔기 개수 (정수)
    """
    try:
        # 안전하게 문자열을 튜플로 변환
        residues = ast.literal_eval(x)
        return len(residues)
    except Exception as e:
        print(f"파싱 에러: {x} -> {e}")
        return None

# "Binding Residues" 열에 대해 결합 잔기 개수를 새 컬럼 'binding_count'에 저장
df['binding_count'] = df['Binding Residues'].apply(count_binding_residues)

# binding_count별로 데이터프레임 행의 개수를 집계
group_df = df.groupby('binding_count').size().reset_index(name='df_count')

print(group_df)


   binding_count  df_count
0              1      1454
1              2     11147
2              3     22054
3              4     41847
4              5     11537
5              6      3675
6              7        42
7              8         9
